<a href="https://colab.research.google.com/github/nannthd/project_AIEngineer/blob/main/AllModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Resnet50

##original

###224

In [2]:
import zipfile
import os

def unzip_file(zip_path, extract_to_folder):
    # ตรวจสอบว่ามีโฟลเดอร์ปลายทางหรือไม่ ถ้าไม่มีก็สร้างใหม่
    if not os.path.exists(extract_to_folder):
        os.makedirs(extract_to_folder)

    # เปิดไฟล์ ZIP และแตกไฟล์
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_folder)
        print(f'ไฟล์ถูกแตกที่ {extract_to_folder}')

# ตัวอย่างการใช้งาน
zip_path = '/content/drive/MyDrive/drug/drug50cls.zip'  # ใส่เส้นทางไฟล์ ZIP ของคุณที่นี่
extract_to_folder = '/content/drug50cls'  # ใส่เส้นทางโฟลเดอร์ที่ต้องการแตกไฟล์ที่นี่
unzip_file(zip_path, extract_to_folder)

ไฟล์ถูกแตกที่ /content/drug50cls


In [ ]:
!pip install tensorflow

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from PIL import Image
import matplotlib.pyplot as plt

# ใช้โมเดล ResNet50 ของ TensorFlow ในการทำ embedding
model = ResNet50(include_top=False, weights='imagenet', pooling='avg')

def image_embedding(path):
    img = Image.open(path)
    img = img.resize((224, 224))  # ปรับขนาดภาพให้เข้ากับ ResNet50
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    curr_df = pd.DataFrame(preds[0]).T
    return curr_df

def load_embeddings_from_csv(csv_folder):
    all_embeddings = {}
    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            class_name = os.path.splitext(csv_file)[0]  # ใช้ชื่อไฟล์ CSV เป็นชื่อคลาส
            csv_path = os.path.join(csv_folder, csv_file)
            df = pd.read_csv(csv_path)
            embeddings_no_id = df.drop(['ID'], axis=1)
            all_embeddings[class_name] = embeddings_no_id
    return all_embeddings

def find_most_similar_classes(new_image_path, all_embeddings):
    new_embedding = image_embedding(new_image_path)
    similarity_scores = {}

    for class_name, embeddings_df in all_embeddings.items():
        similarity_score = cosine_similarity(new_embedding, embeddings_df)
        max_similarity_score = similarity_score.max()  # หา similarity ที่สูงที่สุดในคลาส
        similarity_scores[class_name] = max_similarity_score

    sorted_similarity = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
    top_5_similar_classes = sorted_similarity[:5]

    return top_5_similar_classes

def show_images(image_paths, title):
    plt.figure(figsize=(5, 5))
    plt.suptitle(title, fontsize=16)
    for i, path in enumerate(image_paths):
        img = Image.open(path)
        plt.subplot(1, len(image_paths), i + 1)
        plt.imshow(img)
        plt.axis('off')
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

# โหลด embedding จากโฟลเดอร์ที่เก็บไฟล์ CSV
csv_folder_path = '/content/drug50cls'
all_embeddings = load_embeddings_from_csv(csv_folder_path)

# โฟลเดอร์ที่เก็บภาพที่จะทดสอบ
test_image_folder = '/content/drive/MyDrive/drug/CorpTest'

# ประมวลผลภาพทั้งหมดในโฟลเดอร์
for image_file in os.listdir(test_image_folder):
    if image_file.endswith(('jpg', 'png')):
        image_path = os.path.join(test_image_folder, image_file)

        # ค้นหาคลาสที่มีความคล้ายกันที่สุด 5 อันดับ
        similar_classes = find_most_similar_classes(image_path, all_embeddings)

        # แสดงผลลัพธ์
        print(f"Results for image: {image_file}")
        for rank, (class_name, similarity_score) in enumerate(similar_classes, start=1):
            print(f"{rank}. Class: {class_name}, Similarity Score: {similarity_score}")

        # แสดงภาพทดสอบ
        print("Showing test image:")
        show_images([image_path], "Test Image")

        # เพิ่มภาพที่คล้ายคลึงที่สุดในแต่ละคลาสที่พบ
        image_paths = []
        for class_name, _ in similar_classes:
            class_image_folder = os.path.join(csv_folder_path, class_name)
            if os.path.isdir(class_image_folder):
                image_files = [f for f in os.listdir(class_image_folder) if f.endswith(('jpg', 'png'))]
                if image_files:
                    top_image_path = os.path.join(class_image_folder, image_files[0])
                    image_paths.append(top_image_path)

        # แสดงภาพจากคลาสที่คล้ายคลึงที่สุด
        if image_paths:
            print("Showing similar images:")
            show_images(image_paths, "Similar Images")

        print("\n" + "="*40 + "\n")

ผิด1

###640

In [17]:
import zipfile
import os

def unzip_file(zip_path, extract_to_folder):
    # ตรวจสอบว่ามีโฟลเดอร์ปลายทางหรือไม่ ถ้าไม่มีก็สร้างใหม่
    if not os.path.exists(extract_to_folder):
        os.makedirs(extract_to_folder)

    # เปิดไฟล์ ZIP และแตกไฟล์
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_folder)
        print(f'ไฟล์ถูกแตกที่ {extract_to_folder}')

# ตัวอย่างการใช้งาน
zip_path = '/content/drive/MyDrive/drug/drug50cls_640.zip'  # ใส่เส้นทางไฟล์ ZIP ของคุณที่นี่
extract_to_folder = '/content/drug50cls_640'  # ใส่เส้นทางโฟลเดอร์ที่ต้องการแตกไฟล์ที่นี่
unzip_file(zip_path, extract_to_folder)

ไฟล์ถูกแตกที่ /content/drug50cls_640


In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from PIL import Image
import matplotlib.pyplot as plt

# ใช้โมเดล ResNet50 ของ TensorFlow ในการทำ embedding
model = ResNet50(include_top=False, weights='imagenet', pooling='avg')

def image_embedding(path):
    img = Image.open(path)
    img = img.resize((640, 640))  # ปรับขนาดภาพให้เข้ากับ ResNet50
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    preds = model.predict(x)
    curr_df = pd.DataFrame(preds[0]).T
    return curr_df

def load_embeddings_from_csv(csv_folder):
    all_embeddings = {}
    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            class_name = os.path.splitext(csv_file)[0]  # ใช้ชื่อไฟล์ CSV เป็นชื่อคลาส
            csv_path = os.path.join(csv_folder, csv_file)
            df = pd.read_csv(csv_path)
            embeddings_no_id = df.drop(['ID'], axis=1)
            all_embeddings[class_name] = embeddings_no_id
    return all_embeddings

def find_most_similar_classes(new_image_path, all_embeddings):
    new_embedding = image_embedding(new_image_path)
    similarity_scores = {}

    for class_name, embeddings_df in all_embeddings.items():
        similarity_score = cosine_similarity(new_embedding, embeddings_df)
        max_similarity_score = similarity_score.max()  # หา similarity ที่สูงที่สุดในคลาส
        similarity_scores[class_name] = max_similarity_score

    sorted_similarity = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
    top_5_similar_classes = sorted_similarity[:5]

    return top_5_similar_classes

def show_images(image_paths, title):
    plt.figure(figsize=(5, 5))
    plt.suptitle(title, fontsize=16)
    for i, path in enumerate(image_paths):
        img = Image.open(path)
        plt.subplot(1, len(image_paths), i + 1)
        plt.imshow(img)
        plt.axis('off')
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

# โหลด embedding จากโฟลเดอร์ที่เก็บไฟล์ CSV
csv_folder_path = '/content/drug50cls_640'
all_embeddings = load_embeddings_from_csv(csv_folder_path)

# โฟลเดอร์ที่เก็บภาพที่จะทดสอบ
test_image_folder = '/content/drive/MyDrive/drug/CorpTest'

# ประมวลผลภาพทั้งหมดในโฟลเดอร์
for image_file in os.listdir(test_image_folder):
    if image_file.endswith(('jpg', 'png')):
        image_path = os.path.join(test_image_folder, image_file)

        # ค้นหาคลาสที่มีความคล้ายกันที่สุด 5 อันดับ
        similar_classes = find_most_similar_classes(image_path, all_embeddings)

        # แสดงผลลัพธ์
        print(f"Results for image: {image_file}")
        for rank, (class_name, similarity_score) in enumerate(similar_classes, start=1):
            print(f"{rank}. Class: {class_name}, Similarity Score: {similarity_score}")

        # แสดงภาพทดสอบ
        print("Showing test image:")
        show_images([image_path], "Test Image")

        # เพิ่มภาพที่คล้ายคลึงที่สุดในแต่ละคลาสที่พบ
        image_paths = []
        for class_name, _ in similar_classes:
            class_image_folder = os.path.join(csv_folder_path, class_name)
            if os.path.isdir(class_image_folder):
                image_files = [f for f in os.listdir(class_image_folder) if f.endswith(('jpg', 'png'))]
                if image_files:
                    top_image_path = os.path.join(class_image_folder, image_files[0])
                    image_paths.append(top_image_path)

        # แสดงภาพจากคลาสที่คล้ายคลึงที่สุด
        if image_paths:
            print("Showing similar images:")
            show_images(image_paths, "Similar Images")

        print("\n" + "="*40 + "\n")

ผิด3

##Fine Tune

###224

In [19]:
import zipfile
import os

def unzip_file(zip_path, extract_to_folder):
    # ตรวจสอบว่ามีโฟลเดอร์ปลายทางหรือไม่ ถ้าไม่มีก็สร้างใหม่
    if not os.path.exists(extract_to_folder):
        os.makedirs(extract_to_folder)

    # เปิดไฟล์ ZIP และแตกไฟล์
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_folder)
        print(f'ไฟล์ถูกแตกที่ {extract_to_folder}')

# ตัวอย่างการใช้งาน
zip_path = '/content/drive/MyDrive/drug/drug50cls_ResNet50_FT_model_v1.zip'  # ใส่เส้นทางไฟล์ ZIP ของคุณที่นี่
extract_to_folder = '/content/drug50cls_ResNet50_FT_model_v1'  # ใส่เส้นทางโฟลเดอร์ที่ต้องการแตกไฟล์ที่นี่
unzip_file(zip_path, extract_to_folder)

ไฟล์ถูกแตกที่ /content/drug50cls_ResNet50_FT_model_v1


In [ ]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define a feature extractor model
class FeatureExtractor(nn.Module):
    def __init__(self, base_model):
        super(FeatureExtractor, self).__init__()
        self.base_model = base_model
        self.base_model.fc = nn.Identity()  # Remove the final classification layer

    def forward(self, x):
        return self.base_model(x)

# Initialize ResNet-50 model and feature extractor
resnet50_model = models.resnet50(pretrained=False)
num_classes = 50  # Replace with the actual number of classes from your training
resnet50_model.fc = nn.Linear(resnet50_model.fc.in_features, num_classes)  # Replace the final layer with your trained layer
model_path = '/content/drive/MyDrive/drug/model_resnet50.pth'
resnet50_model.load_state_dict(torch.load(model_path))
feature_extractor = FeatureExtractor(resnet50_model).to(device)
feature_extractor.eval()  # Set the model to evaluation mode

# Define the image transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

def image_embedding(path):
    img = Image.open(path).convert('RGB')  # Convert image to RGB
    img = transform(img).unsqueeze(0).to(device)  # Apply transformation and add batch dimension

    with torch.no_grad():
        # Extract features from the penultimate layer
        features = feature_extractor(img)
        avg_embedding = features.squeeze().cpu().numpy()  # Convert to numpy array and remove batch dimension

    curr_df = pd.DataFrame(avg_embedding).T
    return curr_df

def load_embeddings_from_csv(csv_folder):
    all_embeddings = {}
    if not os.path.exists(csv_folder):
        raise FileNotFoundError(f"CSV folder {csv_folder} not found.")

    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            class_name = os.path.splitext(csv_file)[0]  # Use CSV file name as class name
            csv_path = os.path.join(csv_folder, csv_file)
            df = pd.read_csv(csv_path)
            if 'ID' in df.columns:
                # Drop ID column and keep only class embeddings
                embeddings_no_id = df.drop(['ID'], axis=1)
                all_embeddings[class_name] = embeddings_no_id
            else:
                raise ValueError(f"ID column not found in {csv_file}.")
    return all_embeddings

def find_most_similar_classes(new_image_path, all_embeddings):
    if not os.path.isdir(new_image_path):
        raise FileNotFoundError(f"Directory {new_image_path} not found.")

    image_files = [os.path.join(new_image_path, f) for f in os.listdir(new_image_path) if f.lower().endswith(('png', 'jpg', 'jpeg'))]
    results = {}

    for image_path in image_files:
        new_embedding = image_embedding(image_path)
        similarity_scores = {}

        for class_name, embeddings_df in all_embeddings.items():
            # Compute similarity between new image embedding and class embeddings
            similarity_score = cosine_similarity(new_embedding, embeddings_df)
            max_similarity_score = similarity_score.max()  # Get highest similarity score in class
            similarity_scores[class_name] = max_similarity_score

        sorted_similarity = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
        top_5_similar_classes = sorted_similarity[:5]

        results[image_path] = top_5_similar_classes

    return results

def display_image_with_results(image_path, similar_classes):
    img = Image.open(image_path)
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Top 5 Similar Classes for {os.path.basename(image_path)}")
    plt.show()

# Load embeddings from CSV folder
csv_folder_path = '/content/drug50cls_ResNet50_FT_model_v1'
try:
    all_embeddings = load_embeddings_from_csv(csv_folder_path)
except (FileNotFoundError, ValueError) as e:
    print(e)
    all_embeddings = {}

# Directory containing images to test
new_image_folder_path = '/content/drive/MyDrive/drug/CorpTest'

if all_embeddings:
    # Find top 5 most similar classes for each image in the folder
    try:
        similar_classes_results = find_most_similar_classes(new_image_folder_path, all_embeddings)
        # Print results and display images
        for image_path, similar_classes in similar_classes_results.items():
            print(f"\nImage: {image_path}")
            print("Top 5 most similar classes:")
            for rank, (class_name, similarity_score) in enumerate(similar_classes, start=1):
                print(f"{rank}. Class: {class_name}, Similarity Score: {similarity_score}")
            display_image_with_results(image_path, similar_classes)
    except FileNotFoundError as e:
        print(e)
else:
    print("No embeddings loaded, cannot perform similarity search.")

ผิด1

###640

In [21]:
import zipfile
import os

def unzip_file(zip_path, extract_to_folder):
    # ตรวจสอบว่ามีโฟลเดอร์ปลายทางหรือไม่ ถ้าไม่มีก็สร้างใหม่
    if not os.path.exists(extract_to_folder):
        os.makedirs(extract_to_folder)

    # เปิดไฟล์ ZIP และแตกไฟล์
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_folder)
        print(f'ไฟล์ถูกแตกที่ {extract_to_folder}')

# ตัวอย่างการใช้งาน
zip_path = '/content/drive/MyDrive/drug/drug50cls_ResNet50_FT_model_v1_640.zip'  # ใส่เส้นทางไฟล์ ZIP ของคุณที่นี่
extract_to_folder = '/content/drug50cls_ResNet50_FT_model_v1_640'  # ใส่เส้นทางโฟลเดอร์ที่ต้องการแตกไฟล์ที่นี่
unzip_file(zip_path, extract_to_folder)

ไฟล์ถูกแตกที่ /content/drug50cls_ResNet50_FT_model_v1_640


In [ ]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

# Set up device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define a feature extractor model
class FeatureExtractor(nn.Module):
    def __init__(self, base_model):
        super(FeatureExtractor, self).__init__()
        self.base_model = base_model
        self.base_model.fc = nn.Identity()  # Remove the final classification layer

    def forward(self, x):
        return self.base_model(x)

# Initialize ResNet-50 model and feature extractor
resnet50_model = models.resnet50(pretrained=False)
num_classes = 50  # Replace with the actual number of classes from your training
resnet50_model.fc = nn.Linear(resnet50_model.fc.in_features, num_classes)  # Replace the final layer with your trained layer
model_path = '/content/drive/MyDrive/drug/trained_resnet50_epoch_3.pth'
resnet50_model.load_state_dict(torch.load(model_path))
feature_extractor = FeatureExtractor(resnet50_model).to(device)
feature_extractor.eval()  # Set the model to evaluation mode

# Define the image transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

def image_embedding(path):
    img = Image.open(path).convert('RGB')  # Convert image to RGB
    img = transform(img).unsqueeze(0).to(device)  # Apply transformation and add batch dimension

    with torch.no_grad():
        # Extract features from the penultimate layer
        features = feature_extractor(img)
        avg_embedding = features.squeeze().cpu().numpy()  # Convert to numpy array and remove batch dimension

    curr_df = pd.DataFrame(avg_embedding).T
    return curr_df

def load_embeddings_from_csv(csv_folder):
    all_embeddings = {}
    if not os.path.exists(csv_folder):
        raise FileNotFoundError(f"CSV folder {csv_folder} not found.")

    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            class_name = os.path.splitext(csv_file)[0]  # Use CSV file name as class name
            csv_path = os.path.join(csv_folder, csv_file)
            df = pd.read_csv(csv_path)
            if 'ID' in df.columns:
                # Drop ID column and keep only class embeddings
                embeddings_no_id = df.drop(['ID'], axis=1)
                all_embeddings[class_name] = embeddings_no_id
            else:
                raise ValueError(f"ID column not found in {csv_file}.")
    return all_embeddings

def find_most_similar_classes(new_image_path, all_embeddings):
    if not os.path.isdir(new_image_path):
        raise FileNotFoundError(f"Directory {new_image_path} not found.")

    image_files = [os.path.join(new_image_path, f) for f in os.listdir(new_image_path) if f.lower().endswith(('png', 'jpg', 'jpeg'))]
    results = {}

    for image_path in image_files:
        new_embedding = image_embedding(image_path)
        similarity_scores = {}

        for class_name, embeddings_df in all_embeddings.items():
            # Compute similarity between new image embedding and class embeddings
            similarity_score = cosine_similarity(new_embedding, embeddings_df)
            max_similarity_score = similarity_score.max()  # Get highest similarity score in class
            similarity_scores[class_name] = max_similarity_score

        sorted_similarity = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
        top_5_similar_classes = sorted_similarity[:5]

        results[image_path] = top_5_similar_classes

    return results

def display_image_with_results(image_path, similar_classes):
    img = Image.open(image_path)
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Top 5 Similar Classes for {os.path.basename(image_path)}")
    plt.show()

# Load embeddings from CSV folder
csv_folder_path = '/content/drug50cls_ResNet50_FT_model_v1_640'
try:
    all_embeddings = load_embeddings_from_csv(csv_folder_path)
except (FileNotFoundError, ValueError) as e:
    print(e)
    all_embeddings = {}

# Directory containing images to test
new_image_folder_path = '/content/drive/MyDrive/drug/CorpTest'

if all_embeddings:
    # Find top 5 most similar classes for each image in the folder
    try:
        similar_classes_results = find_most_similar_classes(new_image_folder_path, all_embeddings)
        # Print results and display images
        for image_path, similar_classes in similar_classes_results.items():
            print(f"\nImage: {image_path}")
            print("Top 5 most similar classes:")
            for rank, (class_name, similarity_score) in enumerate(similar_classes, start=1):
                print(f"{rank}. Class: {class_name}, Similarity Score: {similarity_score}")
            display_image_with_results(image_path, similar_classes)
    except FileNotFoundError as e:
        print(e)
else:
    print("No embeddings loaded, cannot perform similarity search.")

ผิด26

#ViTModel

##original

###224

In [29]:
import zipfile
import os

def unzip_file(zip_path, extract_to_folder):
    # ตรวจสอบว่ามีโฟลเดอร์ปลายทางหรือไม่ ถ้าไม่มีก็สร้างใหม่
    if not os.path.exists(extract_to_folder):
        os.makedirs(extract_to_folder)

    # เปิดไฟล์ ZIP และแตกไฟล์
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_folder)
        print(f'ไฟล์ถูกแตกที่ {extract_to_folder}')

# ตัวอย่างการใช้งาน
zip_path = '/content/drive/MyDrive/drug/drug50cls_ViTModel.zip'  # ใส่เส้นทางไฟล์ ZIP ของคุณที่นี่
extract_to_folder = '/content/drug50cls_ViTModel'  # ใส่เส้นทางโฟลเดอร์ที่ต้องการแตกไฟล์ที่นี่
unzip_file(zip_path, extract_to_folder)

ไฟล์ถูกแตกที่ /content/drug50cls_ViTModel


In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import torch
import matplotlib.pyplot as plt

# ใช้โมเดล ViT ของ Hugging Face ในการทำ embedding
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

# ย้ายโมเดลไปยัง GPU ถ้ามี
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def image_embedding(path):
    try:
        img = Image.open(path).convert('RGB').resize((224, 224))  # ปรับขนาดภาพให้เข้ากับ ViT
        inputs = processor(images=img, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        last_hidden_states = outputs.last_hidden_state
        avg_embedding = last_hidden_states.mean(dim=1).squeeze().cpu().detach().numpy()  # ค่าเฉลี่ยของ hidden states
        curr_df = pd.DataFrame(avg_embedding).T
        return curr_df
    except Exception as e:
        print(f"Error processing image {path}: {e}")
        return pd.DataFrame()

def load_embeddings_from_csv(csv_folder):
    all_embeddings = {}
    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            class_name = os.path.splitext(csv_file)[0]  # ใช้ชื่อไฟล์ CSV เป็นชื่อคลาส
            csv_path = os.path.join(csv_folder, csv_file)
            df = pd.read_csv(csv_path)
            # ดรอปคอลัมน์ ID และเก็บเฉพาะเวกเตอร์ของคลาส
            embeddings_no_id = df.drop(['ID'], axis=1)
            all_embeddings[class_name] = embeddings_no_id
    return all_embeddings

def find_most_similar_classes(new_image_path, all_embeddings):
    new_embedding = image_embedding(new_image_path)
    similarity_scores = {}

    for class_name, embeddings_df in all_embeddings.items():
        # คำนวณค่า similarity ระหว่าง embedding ของรูปภาพใหม่กับ embeddings ของคลาส
        similarity_score = cosine_similarity(new_embedding, embeddings_df)
        max_similarity_score = similarity_score.max()  # หา similarity ที่สูงที่สุดในคลาส
        similarity_scores[class_name] = max_similarity_score

    sorted_similarity = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
    top_5_similar_classes = sorted_similarity[:5]

    return top_5_similar_classes

def process_images_in_folder(image_folder_path, all_embeddings):
    results = {}
    for image_file in os.listdir(image_folder_path):
        image_path = os.path.join(image_folder_path, image_file)
        if image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            print(f"Processing image: {image_file}")
            similar_classes = find_most_similar_classes(image_path, all_embeddings)
            results[image_file] = (image_path, similar_classes)
    return results

def display_images_with_results(results):
    for image_file, (image_path, similar_classes) in results.items():
        img = Image.open(image_path)
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 1, 1)
        plt.imshow(img)
        plt.title(f"Image: {image_file}")
        plt.axis('off')

        print(f"\nImage: {image_file}")
        print("Top 5 most similar classes:")
        for rank, (class_name, similarity_score) in enumerate(similar_classes, start=1):
            print(f"{rank}. Class: {class_name}, Similarity Score: {similarity_score}")

        plt.show()

# โหลด embedding จากโฟลเดอร์ที่เก็บไฟล์ CSV
csv_folder_path = '/content/drug50cls_ViTModel'
all_embeddings = load_embeddings_from_csv(csv_folder_path)

# โฟลเดอร์ที่เก็บรูปภาพที่ต้องการทดสอบ
image_folder_path = '/content/drive/MyDrive/drug/CorpTest'

# ค้นหาคลาสที่มีความคล้ายกันที่สุดสำหรับแต่ละภาพในโฟลเดอร์
results = process_images_in_folder(image_folder_path, all_embeddings)

# แสดงภาพพร้อมกับผลลัพธ์
display_images_with_results(results)

###640

In [31]:
import zipfile
import os

def unzip_file(zip_path, extract_to_folder):
    # ตรวจสอบว่ามีโฟลเดอร์ปลายทางหรือไม่ ถ้าไม่มีก็สร้างใหม่
    if not os.path.exists(extract_to_folder):
        os.makedirs(extract_to_folder)

    # เปิดไฟล์ ZIP และแตกไฟล์
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_folder)
        print(f'ไฟล์ถูกแตกที่ {extract_to_folder}')

# ตัวอย่างการใช้งาน
zip_path = '/content/drive/MyDrive/drug/drug50cls_ViTModel_640.zip'  # ใส่เส้นทางไฟล์ ZIP ของคุณที่นี่
extract_to_folder = '/content/drug50cls_ViTModel_640'  # ใส่เส้นทางโฟลเดอร์ที่ต้องการแตกไฟล์ที่นี่
unzip_file(zip_path, extract_to_folder)

ไฟล์ถูกแตกที่ /content/drug50cls_ViTModel_640


In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import torch
import matplotlib.pyplot as plt

# ใช้โมเดล ViT ของ Hugging Face ในการทำ embedding
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

# ย้ายโมเดลไปยัง GPU ถ้ามี
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def image_embedding(path):
    try:
        img = Image.open(path).convert('RGB').resize((640, 640))  # ปรับขนาดภาพให้เข้ากับ ViT
        inputs = processor(images=img, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        last_hidden_states = outputs.last_hidden_state
        avg_embedding = last_hidden_states.mean(dim=1).squeeze().cpu().detach().numpy()  # ค่าเฉลี่ยของ hidden states
        curr_df = pd.DataFrame(avg_embedding).T
        return curr_df
    except Exception as e:
        print(f"Error processing image {path}: {e}")
        return pd.DataFrame()

def load_embeddings_from_csv(csv_folder):
    all_embeddings = {}
    for csv_file in os.listdir(csv_folder):
        if csv_file.endswith('.csv'):
            class_name = os.path.splitext(csv_file)[0]  # ใช้ชื่อไฟล์ CSV เป็นชื่อคลาส
            csv_path = os.path.join(csv_folder, csv_file)
            df = pd.read_csv(csv_path)
            # ดรอปคอลัมน์ ID และเก็บเฉพาะเวกเตอร์ของคลาส
            embeddings_no_id = df.drop(['ID'], axis=1)
            all_embeddings[class_name] = embeddings_no_id
    return all_embeddings

def find_most_similar_classes(new_image_path, all_embeddings):
    new_embedding = image_embedding(new_image_path)
    similarity_scores = {}

    for class_name, embeddings_df in all_embeddings.items():
        # คำนวณค่า similarity ระหว่าง embedding ของรูปภาพใหม่กับ embeddings ของคลาส
        similarity_score = cosine_similarity(new_embedding, embeddings_df)
        max_similarity_score = similarity_score.max()  # หา similarity ที่สูงที่สุดในคลาส
        similarity_scores[class_name] = max_similarity_score

    sorted_similarity = sorted(similarity_scores.items(), key=lambda x: x[1], reverse=True)
    top_5_similar_classes = sorted_similarity[:5]

    return top_5_similar_classes

def process_images_in_folder(image_folder_path, all_embeddings):
    results = {}
    for image_file in os.listdir(image_folder_path):
        image_path = os.path.join(image_folder_path, image_file)
        if image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            print(f"Processing image: {image_file}")
            similar_classes = find_most_similar_classes(image_path, all_embeddings)
            results[image_file] = (image_path, similar_classes)
    return results

def display_images_with_results(results):
    for image_file, (image_path, similar_classes) in results.items():
        img = Image.open(image_path)
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 1, 1)
        plt.imshow(img)
        plt.title(f"Image: {image_file}")
        plt.axis('off')

        print(f"\nImage: {image_file}")
        print("Top 5 most similar classes:")
        for rank, (class_name, similarity_score) in enumerate(similar_classes, start=1):
            print(f"{rank}. Class: {class_name}, Similarity Score: {similarity_score}")

        plt.show()

# โหลด embedding จากโฟลเดอร์ที่เก็บไฟล์ CSV
csv_folder_path = '/content/drug50cls_ViTModel_640'
all_embeddings = load_embeddings_from_csv(csv_folder_path)

# โฟลเดอร์ที่เก็บรูปภาพที่ต้องการทดสอบ
image_folder_path = '/content/drive/MyDrive/drug/CorpTest'

# ค้นหาคลาสที่มีความคล้ายกันที่สุดสำหรับแต่ละภาพในโฟลเดอร์
results = process_images_in_folder(image_folder_path, all_embeddings)

# แสดงภาพพร้อมกับผลลัพธ์
display_images_with_results(results)

##Fine Tune

###224

In [33]:
import zipfile
import os

def unzip_file(zip_path, extract_to_folder):
    # ตรวจสอบว่ามีโฟลเดอร์ปลายทางหรือไม่ ถ้าไม่มีก็สร้างใหม่
    if not os.path.exists(extract_to_folder):
        os.makedirs(extract_to_folder)

    # เปิดไฟล์ ZIP และแตกไฟล์
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_folder)
        print(f'ไฟล์ถูกแตกที่ {extract_to_folder}')

# ตัวอย่างการใช้งาน
zip_path = '/content/drive/MyDrive/drug/drug50cls_ViT-FT-model-v1.zip'  # ใส่เส้นทางไฟล์ ZIP ของคุณที่นี่
extract_to_folder = '/content/drug50cls_ViT-FT-model-v1'  # ใส่เส้นทางโฟลเดอร์ที่ต้องการแตกไฟล์ที่นี่
unzip_file(zip_path, extract_to_folder)

ไฟล์ถูกแตกที่ /content/drug50cls_ViT-FT-model-v1


In [ ]:
import zipfile
import os

def unzip_file(zip_path, extract_to_folder):
    # ตรวจสอบว่ามีโฟลเดอร์ปลายทางหรือไม่ ถ้าไม่มีก็สร้างใหม่
    if not os.path.exists(extract_to_folder):
        os.makedirs(extract_to_folder)

    # เปิดไฟล์ ZIP และแตกไฟล์
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_folder)
        print(f'ไฟล์ถูกแตกที่ {extract_to_folder}')

# ตัวอย่างการใช้งาน
zip_path = '/content/drive/MyDrive/drug/drug50cls_ViT-FT-model-v1.zip'  # ใส่เส้นทางไฟล์ ZIP ของคุณที่นี่
extract_to_folder = '/content/drug50cls_ViT-FT-model-v1'  # ใส่เส้นทางโฟลเดอร์ที่ต้องการแตกไฟล์ที่นี่
unzip_file(zip_path, extract_to_folder)

###640